In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import dataloader
from torch.utils.data import Dataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from IPython.display import display
import torch.optim as optim

import h5py
import sys
sys.path.append('../')
from utils.common_fcns import *
import matplotlib
from matplotlib import pyplot as plt

/home/alex/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#device = torch.device('cuda')

### Load Data

In [3]:
# Uncomment the below line if you haven't created the dataset file yet
#!python prepare_training_data.py 

# Load in the data file
df = pd.read_pickle('../data/training_set_true_image.pkl')
display(df)

class Data(Dataset):
    def __init__(self, x, y):
        super(Data, self).__init__()
        self.x = x
        self.y = y

    def __len__(self):
        return self.y.shape[0]

    def __getitem__(self, index):
        return self.x[index], self.y[index]

train_slices = df['slice'].to_numpy()
data = Data(train_slices,train_slices)

,slice_name,slice
0,file_brain_AXT2_200_2000535_0,"[1.0057089e-05, 9.772637e-06, 1.2455052e-05, 1..."
1,file_brain_AXT2_200_2000535_1,"[1.37167e-05, 1.5023721e-05, 1.3295172e-05, 1...."
2,file_brain_AXT2_200_2000535_2,"[1.39793265e-05, 2.2203161e-05, 1.8096427e-05,..."
3,file_brain_AXT2_200_2000535_3,"[1.4201461e-05, 1.3245278e-05, 1.45633185e-05,..."
4,file_brain_AXT2_200_2000535_4,"[6.0453976e-06, 9.784446e-06, 7.2580615e-06, 7..."
...,...,...
425,file_brain_AXT2_200_2000368_11,"[5.3172553e-06, 5.5633427e-06, 6.050383e-06, 7..."
426,file_brain_AXT2_200_2000368_12,"[6.7074516e-06, 6.0870066e-06, 8.798891e-06, 6..."
427,file_brain_AXT2_200_2000368_13,"[7.124069e-06, 7.744692e-06, 7.642781e-06, 4.7..."
428,file_brain_AXT2_200_2000368_14,"[7.2822577e-06, 5.6842214e-06, 6.9956786e-06, ..."


### Define the Neural Network

In [4]:
# Inherit from torch.nn, define basic k-latent AutoEncoder
k_latent = 3000

class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()
        self.k_latent_encode = nn.Linear(256*256, k_latent)
        self.k_latent_decode = nn.Linear(k_latent, 256*256)

    def forward(self, x):
        x = self.k_latent_encode(x)
        x = torch.relu(x)
        x = self.k_latent_decode(x)
        x = torch.sigmoid(x)
        return x

### Train the Model

In [5]:
# Training setup
n_epochs = 100
minibatch_size = 429 #Full batch
batch_loss = np.zeros(n_epochs)

model = AutoEncoder()
#model.to(device)
optimizer = optim.SGD(model.parameters(), lr=100000)
criterion = nn.MSELoss()

In [6]:
# Do training
for epoch in range(n_epochs):
    # Pick batches for this epoch
    train_loader = torch.utils.data.DataLoader(data, batch_size=minibatch_size, shuffle=True)
    for batch_idx, (X, _) in enumerate(train_loader):
        optimizer.zero_grad()   # zero the gradient buffers
        Y = model(X)
        loss = criterion(X,Y)
        batch_loss[epoch]+=loss.item() # save off the loss
        loss.backward()
        optimizer.step()    # update
    print("Completed epoch: "+str(epoch)+" , Loss: "+str(batch_loss[epoch]))

Completed epoch: 0 , Loss: 0.345411092042923
Completed epoch: 1 , Loss: 3.7955553366941785e-08


### Test It

In [ ]:
# Load a scan in
data_dir = '../data/multicoil_test/'
file = data_dir+'file_brain_AXT2_200_2000482.h5'
hf = h5py.File(file)
volume_kspace = hf['kspace'][()]
n_slices = volume_kspace.shape[0]
X_raw = combine_all_coils(volume_kspace,8)
X = resize_scan(X_raw)

In [ ]:
# Put it through our autoencoder and see what we get!
X_input = torch.from_numpy(X.flatten())
X_output = model(X_input)
X_output = X_output.detach().numpy().reshape(256,256)
plt.imshow(np.abs(X_output), cmap='gray')